In [ ]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
import os

class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing
        self.width = self.disp.width
        self.height = self.disp.height

    def show_opening_animation(self):
        opening_frames = []
        for i in range(1, 4):  # opening1.jpeg, opening2.jpeg, opening3.jpeg
            frame_path = f"/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/opening/opening{i}.jpeg"  # 경로 수정
            try:
                frame_image = Image.open(frame_path)
                opening_frames.append(frame_image)
            except FileNotFoundError:
                print(f"Error: {frame_path} not found.")
                return  # 파일이 없으면 메서드 종료

        for frame in opening_frames:
            resized_frame = frame.resize((self.disp.width, self.disp.height))
            self.disp.image(resized_frame)
            time.sleep(0.5)
        time.sleep(1)  # 오프닝 후 대기


class Character:
    def __init__(self, width, height):
        self.appearance = 'image'
        self.state = None
        self.width = width
        self.height = height
        self.size_factor = 1.0  # 기본 크기 비율
        
        # 기본 속성 설정
        self.position = np.array([width / 2 - 15, height / 2 - 15, width / 2 + 15, height / 2 + 15])
        self.outline = "#FFFFFF"
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        
        # 디폴트 이미지
        self.image_paths = {
            "right": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-right.png",
            "left": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-left.png",
            "up": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character_up.png",
            "down": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-right.png",
        }

        # 디폴트 방향으로 이미지를 로드합니다.
        self.direction = "right"
        self.image = self.load_image(self.image_paths[self.direction])

        self.bullets = []  # 총알 목록 초기화

    def load_image(self, path, size_factor=1.0):
        if os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((int(30 * size_factor), int(30 * size_factor)))
        else:
            print(f"Error: Character image not found at {path}")
            return None  # 이미지가 없을 경우 None으로 설정

    def apply_jump_effect(self):
        original_image = self.image  # 현재 이미지 저장
        jump_frames = []  # 점프 모션 프레임 리스트
    
        # 점프 애니메이션 프레임을 생성
        for scale in [1.0, 1.5, 2.0, 1.5, 1.0]:  # 크기를 변화시키는 비율
            frame = self.load_image(self.image_paths[self.direction], size_factor=scale)
            jump_frames.append(frame)
    
        # 점프 모션 프레임을 순서대로 화면에 띄우기
        for frame in jump_frames:
            self.image = frame
            time.sleep(0.1)  # 각 프레임의 표시 시간을 조정 (0.1초)
        
        self.image = original_image  # 원래 이미지로 복원

    
   

    def move(self, command=None, obstacles=None):
        if command and command['move']:
            new_position = self.position.copy()  # 이동할 새로운 위치를 복사
            
            # 이동 방향에 따라 위치를 조정
            if command['up_pressed']:
                new_position[1] -= 5
                new_position[3] -= 5
                self.direction = "up"
            elif command['down_pressed']:
                new_position[1] += 5
                new_position[3] += 5
                self.direction = "down"
            elif command['left_pressed']:
                new_position[0] -= 5
                new_position[2] -= 5
                self.direction = "left"
            elif command['right_pressed']:
                new_position[0] += 5
                new_position[2] += 5
                self.direction = "right"

            # 경계를 체크하면서 이동
            if (new_position[0] >= 0 and new_position[2] <= self.width and 
                new_position[1] >= 0 and new_position[3] <= self.height):
                # 장애물 체크
                if not self.check_collision(new_position, obstacles):
                    self.position = new_position  # 이동이 안전하면 위치 업데이트

            # 이미지 로드
            self.image = self.load_image(self.image_paths[self.direction])

        # Center 업데이트
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

    
    def check_collision(self, new_position, obstacles):
        for obstacle in obstacles:
            if (new_position[0] < obstacle[2] and new_position[2] > obstacle[0] and
                new_position[1] < obstacle[3] and new_position[3] > obstacle[1]):
                return True  # 충돌 발생
        return False  # 충돌 없을 경우

    def upgrade_bullet(self):
        self.has_upgraded_bullet = True  # 업그레이드 상태를 True로 변경
    
    def shoot(self, direction):
        bullet_position = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        bullet = Bullet(bullet_position, direction)
        self.bullets.append(bullet)  # 총알을 목록에 추가
    
class Monster:
    def __init__(self, current_stage):
        self.current_stage = current_stage
        self.position = np.array([50, 50, 80, 80])  # 몬스터의 초기 위치 설정
        self.image = self.load_image()

    def load_image(self):
        if self.current_stage == 1:
            path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/monster/monster_stage1_right.png"
        elif self.current_stage == 2:
            path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/monster/monster_stage2_right.png"
        elif self.current_stage == 3:
            path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/monster/monster_stage3_right.png"
        else:
            path = None  # 추가적인 스테이지에 대한 처리 (예: None)

        if path and os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((50, 50))  # 적당한 크기로 조정
        else:
            print(f"Error: Monster image not found at {path}")
            return None  # 이미지가 없을 경우 None으로 설정

    def draw(self, image):
        if self.image is not None:
            image.paste(self.image, (int(self.position[0]), int(self.position[1])), self.image)  # 이미지 붙여넣기

class Bullet:
    def __init__(self, position, direction, is_upgraded=False):
        self.appearance = 'image'
        self.speed = 10
        self.damage = 10
        self.position = np.array([position[0] - 3, position[1] - 3, position[0] + 3, position[1] + 3])
        self.direction = direction
        self.state = None
        self.outline = "#0000FF"

        # 이미지 경로 설정
        self.image_paths = {
            "up": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_magic_up.png" if is_upgraded else "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_up.png",
            "down": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_magic_down.png" if is_upgraded else "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_down.png",
            "right": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_magic_right.png" if is_upgraded else "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_right.png",
            "left": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_magic_left.png" if is_upgraded else "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/bullet_left.png",
        }
        # 방향에 따라 이미지 로드
        self.image = self.load_image(self.image_paths[self.direction])

    def load_image(self, path):
        if os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((10, 10))  # 적당한 크기로 조정
        else:
            print(f"Error: Bullet image not found at {path}")
            return None  # 이미지가 없을 경우 None으로 설정
        

    def move(self):
        if self.direction == "up":
            self.position[1] -= self.speed
            self.position[3] -= self.speed
        elif self.direction == "down":
            self.position[1] += self.speed
            self.position[3] += self.speed
        elif self.direction == "left":
            self.position[0] -= self.speed
            self.position[2] -= self.speed
        elif self.direction == "right":
            self.position[0] += self.speed
            self.position[2] += self.speed

    def collision_check(self, enemies):
        for enemy in enemies:
            if self.overlap(self.position, enemy.position):
                enemy.state = 'die'
                self.state = 'hit'

    def overlap(self, ego_position, other_position):
        return (ego_position[0] < other_position[2] and
                ego_position[2] > other_position[0] and
                ego_position[1] < other_position[3] and
                ego_position[3] > other_position[1])

class Item:
    def __init__(self, image_path, position):
        self.image_path = image_path  # 아이템 이미지 경로 저장
        self.image = self.load_image(image_path)
        self.position = position  # np.array([x1, y1, x2, y2])

    def load_image(self, path):
        if os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((20, 20))  # 아이템 크기 설정
        else:
            print(f"Error: Item image not found at {path}")
            return None

class Game:
    def __init__(self):
        self.joystick = Joystick()
        self.current_stage = 1  # 기본 스테이지를 1로 설정
        self.background_image = self.load_background_image()
        self.character = Character(self.joystick.width, self.joystick.height)  # 캐릭터 초기화
        self.monster = Monster(self.current_stage)  # 스테이지에 따라 몬스터 생성

        # 하트 이미지 초기화
        self.heart_images = [
            "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/heart/heart_3.png",
            "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/heart/heart_2.png",
            "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/heart/heart_1.png"
        ]
        self.heart_index = 0  # 생명 상태를 관리하는 인덱스 (3일부터 시작)
        self.heart_image = self.load_heart_image()  # 현재 생명에 대한 하트 이미지 로드

        # 캐릭터의 초기 위치를 왼쪽 아래로 설정
        character_start_x = 15   
        character_start_y = self.joystick.height - 45  
        self.character = Character(self.joystick.width, self.joystick.height)
        self.character.position = np.array([character_start_x, character_start_y, character_start_x + 30, character_start_y + 30])

        #스테이지 clear, 게임 clear 이미지
        self.stage_clear_image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage_clear.png"
        self.game_clear_image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage_gameclear.png"
        # 게임 오버 이미지 초기화
        self.game_over_image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage_gameover.png"
        self.game_over_image = None  # 초기값은 None

        # 아이템 초기화
        self.items = [
            Item("/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/item/item_jump.png", np.array([30, 30, 50, 50])),
            Item("/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/item/item_puzzle.png", np.array([100, 100, 120, 120])),
            Item("/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/item/item_upgradeBullet.png", np.array([200, 200, 220, 220])),
        ]

        self.obstacles = [
            np.array([0, 0, 240, 15]),  # 상단 전체
            # 좌측 장애물
            np.array([0, 15, 15, 240]),  # 좌측 전체
            # 하단 장애물
            np.array([0, 225, 240, 240]),  # 하단 전체
            # 우측 장애물
            np.array([225, 15, 240, 240]), # 우측 전체
        ] 
        self.portal_image = self.load_portal_image()  # 문 이미지 로드
        self.set_portal_position()  # 문 위치 설정

        self.acquired_items = []  # 획득한 아이템을 저장할 리스트

    def show_stage_clear_animation(self):
        stage_clear_image = Image.open(self.stage_clear_image_path).convert("RGBA").resize((self.joystick.width, self.joystick.height))
        self.joystick.disp.image(stage_clear_image)
        time.sleep(1)  # 0.5초간 이미지 표시
        self.background_image = self.load_background_image()  # 다음 배경 이미지 로드

    def show_game_clear_animation(self):
        game_clear_image = Image.open(self.game_clear_image_path).convert("RGBA").resize((self.joystick.width, self.joystick.height))
        self.joystick.disp.image(game_clear_image)
        time.sleep(10)  # 게임 클리어 이미지 표시 (2초)

    def load_game_over_image(self):
        if os.path.isfile(self.game_over_image_path):
            return Image.open(self.game_over_image_path).convert("RGBA").resize((self.joystick.width, self.joystick.height))  # 화면 크기에 맞게 조정
        else:
            print(f"Error: Game over image not found at {self.game_over_image_path}")
            return None  # 이미지가 없을 경우 None으로 설정

    def load_portal_image(self):
        portal_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage_portal.png"
        if os.path.isfile(portal_path):
            return Image.open(portal_path).convert("RGBA").resize((30, 40))  # 크기에 맞게 조정
        else:
            print(f"Error: Portal image not found at {portal_path}")
            return None

    def set_portal_position(self):
        if self.current_stage == 1:
            self.portal_position = np.array([self.joystick.width - 90, 0, self.joystick.width - 90, 40])  # 오른쪽 상단에 배치
        elif self.current_stage == 2:
            self.portal_position = np.array([self.joystick.width - 50, 0, self.joystick.width - 10, 40])  # 동일 위치, 필요 시 조정
        elif self.current_stage == 3:
            self.portal_position = np.array([self.joystick.width - 45, 70, self.joystick.width - 60, 60])  # 동일 위치, 필요 시 조정

    def check_portal_collision(self):
        if (self.character.position[0] < self.portal_position[2] and 
            self.character.position[2] > self.portal_position[0] and
            self.character.position[1] < self.portal_position[3] and 
            self.character.position[3] > self.portal_position[1]):
            return True  # 충돌 발생
        return False  # 충돌 없을 경우

    def check_item_collision(self):
        for item in self.items:
            if (self.character.position[0] < item.position[2] and 
                self.character.position[2] > item.position[0] and
                self.character.position[1] < item.position[3] and 
                self.character.position[3] > item.position[1]):
                self.items.remove(item)  # 아이템 삭제

                # 아이템 효과 적용
                if item.image_path == "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/item/item_jump.png":
                    self.character.apply_jump_effect()  # 점프 효과 적용
                
                elif item.image_path == "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/item/item_upgradeBullet.png":
                    self.character.upgrade_bullet()  # 총알 업그레이드 

                self.acquired_items.append(item)  # 획득한 아이템 추가
                break  # 하나의 아이템만 충돌하면 반복 종료

    def load_heart_image(self):
        path = self.heart_images[self.heart_index]
        if os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((60, 30))  # 적당한 크기로 조정
        else:
            print(f"Error: Heart image not found at {path}")
            return None  # 이미지가 없을 경우 None으로 설정
    
    def check_collision(self, position):
        for obstacle in self.obstacles:
            if (position[0] < obstacle[2] and position[2] > obstacle[0] and
                position[1] < obstacle[3] and position[3] > obstacle[1]):
                return True  # 충돌 발생
        return False  # 충돌 없을 경우


    def load_background_image(self):
        # 현재 스테이지에 따라 배경 이미지 경로 설정
        if self.current_stage == 1:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage1.jpeg"
        elif self.current_stage == 2:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage2.jpeg"
        elif self.current_stage == 3:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage3.jpeg"
        else:
            image_path = None

        if image_path is not None and os.path.isfile(image_path):
            return Image.open(image_path).resize((self.joystick.width, self.joystick.height))
        else:
            print(f"Error: {image_path} not found.")
            return None  # 이미지 로드 실패 시 None 반환

    def show_opening_animation(self):
        # Joystick의 show_opening_animation() 호출
        self.joystick.show_opening_animation()
        # 스테이지 변환 코드 추가
        self.background_image = self.load_background_image()  # 스테이지에 맞는 배경 이미지 로드

    def check_monster_collision(self):
        # 캐릭터와 몬스터의 충돌 체크
        if (self.character.position[0] < self.monster.position[2] and 
            self.character.position[2] > self.monster.position[0] and
            self.character.position[1] < self.monster.position[3] and 
            self.character.position[3] > self.monster.position[1]):
            # 충돌 시 캐릭터를 뒤로 5픽셀 튕겨보내기
            if self.character.direction == "up":
                self.character.position[1] += 25
                self.character.position[3] += 25
            elif self.character.direction == "down":
                self.character.position[1] -= 25
                self.character.position[3] -= 25
            elif self.character.direction == "left":
                self.character.position[0] += 25
                self.character.position[2] += 25
            elif self.character.direction == "right":
                self.character.position[0] -= 25
                self.character.position[2] -= 25

            # 하트 감소 처리
            self.heart_index += 1  # 하트 인덱스를 증가
    
            if self.heart_index < len(self.heart_images):
                self.heart_image = self.load_heart_image()  # 하트 이미지 업데이트
            else:
                # 하트 수가 0이면 게임 오버 이미지 표시
                self.game_over_image = self.load_game_over_image()  # 게임 오버 이미지 로드
                if self.game_over_image is not None:
                    self.game_over_image = self.load_game_over_image()  # 게임 오버 이미지 로드
                    return True  # 게임 오버 발생 (True 반환)
                    
            return True  # 충돌 발생
        return False  # 충돌 없을 경우


    def main_loop(self):
        while True:
            my_image = Image.new("RGB", (self.joystick.width, self.joystick.height))
            my_draw = ImageDraw.Draw(my_image)

            # 배경 이미지를 그립니다
            if self.background_image is not None:
                my_image.paste(self.background_image, (0, 0))  # 배경 이미지 붙여 넣기
            else:
                # 배경 이미지가 없을 경우 흰색으로 초기화
                my_draw.rectangle((0, 0, self.joystick.width, self.joystick.height), fill=(255, 255, 255))

            self.monster.draw(my_image)  # monster 이미지 그리기
            
            # 아이템 그리기
            for item in self.items:
                if item.image is not None:
                    item_position = (int(item.position[0]), int(item.position[1]))
                    my_image.paste(item.image, item_position, item.image)
            # 획득한 아이템 그리기
            for idx, acquired_item in enumerate(self.acquired_items):
                if acquired_item.image is not None:
                    item_position = (10, 10 + idx * 25)  # 아래로 25 픽셀 간격으로 위치 설정
                    my_image.paste(acquired_item.image, item_position, acquired_item.image)

            # 하트 이미지 그리기 (왼쪽 하단)
            if self.heart_image is not None:
                heart_position = (0, self.joystick.height - self.heart_image.height)  # 이미지가 왼쪽 하단에 위치하도록 설정
                my_image.paste(self.heart_image, heart_position, self.heart_image)
            
            command = {'move': False, 'up_pressed': False, 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
            # 조이스틱 입력 처리
            if not self.joystick.button_U.value:  # up pressed
                command['up_pressed'] = True
                command['move'] = True
            if not self.joystick.button_D.value:  # down pressed
                command['down_pressed'] = True
                command['move'] = True
            if not self.joystick.button_L.value:  # left pressed
                command['left_pressed'] = True
                command['move'] = True
            if not self.joystick.button_R.value:  # right pressed
                command['right_pressed'] = True
                command['move'] = True
    
            # 총알 발사 처리
            if not self.joystick.button_A.value:  # A 버튼이 눌리면 총알 발사
                direction = self.character.direction  # 현재 방향에 총알 발사
                self.character.shoot(direction)

            # B 버튼 점프 처리
            if not self.joystick.button_B.value:  # B 버튼이 눌리면 점프
                self.character.apply_jump_effect()  # 점프 효과 적용
    
            self.character.move(command, self.obstacles)  # 캐릭터 이동 처리

            # 아이템과의 충돌 체크
            self.check_item_collision()

            # 몬스터와의 충돌 체크
            if self.check_monster_collision():
                if self.heart_index < len(self.heart_images):  # 하트 인덱스가 범위 내인 경우
                    self.heart_image = self.load_heart_image()  # 이미지 업데이트
                else:
                    # 하트 수가 0이면 게임 오버 이미지 표시
                    if self.game_over_image is not None:
                        my_image.paste(self.game_over_image, (0, 0))  # 전체 화면에 게임 오버 이미지 붙여넣기
                        self.joystick.disp.image(my_image)  # 최종 이미지 디스플레이
                    print("Game Over")  # 게임 오버 메시지
                    break  # 게임 루프 중단

    
            # 캐릭터 이미지 그리기
            character_draw_position = (int(self.character.position[0]), int(self.character.position[1]))  # 캐릭터 위치
            if self.character.image is not None:
                my_image.paste(self.character.image, character_draw_position, self.character.image)  # 이미지 붙여넣기
            else:
                my_draw.ellipse(tuple(self.character.position), outline=self.character.outline, fill=(0, 0, 0))  # 기본 원 그리기
    
            # 문 그리기
            if self.portal_image is not None:
                portal_draw_position = (int(self.portal_position[0]), int(self.portal_position[1]))
                my_image.paste(self.portal_image, portal_draw_position, self.portal_image)  # 문 이미지 그리기
    
            # 총알 이동 및 그리기
            for bullet in self.character.bullets:
                bullet.move()  # 총알 이동 처리
                bullet_position = (bullet.position[0], bullet.position[1], bullet.position[2], bullet.position[3])
                if bullet.state != 'hit':
                    if bullet.image is not None:  # 이미지가 있으면 사용
                        bullet_image_position = (int(bullet.position[0]), int(bullet.position[1]))
                        my_image.paste(bullet.image, bullet_image_position, bullet.image)  # 이미지 그리기
                    else:
                        my_draw.rectangle(bullet_position, outline=bullet.outline, fill=(0, 0, 255))  # 이미지가 없을 경우 기본 사각형 그리기
    
            # 문과 캐릭터 충돌 체크
            if self.check_portal_collision():
                if self.current_stage == 1:
                    self.current_stage = 2  # 스테이지를 2로 변경
                    self.show_stage_clear_animation()  # 스테이지 클리어 애니메이션 표시
                    self.set_portal_position()  # 새로운 스테이지의 문 위치 설정
                    self.monster = Monster(self.current_stage)  # 몬스터 업데이트
                elif self.current_stage == 2:
                    self.current_stage = 3  # 스테이지를 3으로 변경
                    self.show_stage_clear_animation()  # 스테이지 클리어 애니메이션 표시
                    self.set_portal_position()  # 새로운 스테이지의 문 위치 설정
                    self.monster = Monster(self.current_stage)  # 몬스터 업데이트
                elif self.current_stage == 3:
                    # 스테이지 3의 포탈 도착 처리
                    self.show_game_clear_animation()  # 게임 클리어 애니메이션 표시

                # 각 스테이지 시작 시 캐릭터 위치 초기화
                character_start_x = 15  # 왼쪽으로 약간 이동
                character_start_y = self.joystick.height - 45  # 화면 하단
                self.character.position = np.array([character_start_x, character_start_y, character_start_x + 30, character_start_y + 30])
                
            # 화면에 현재 이미지 디스플레이
            self.joystick.disp.image(my_image)


def main():
    game = Game()  # Game 객체 생성
    game.show_opening_animation()  # 오프닝 애니메이션 표시 및 스테이지 로드
    game.main_loop()  # 메인 루프 시작

if __name__ == "__main__":
    main()  # 메인 함수 실행